# Comparison between pyBKT and BKT with pyAgrum
In this notebook, we want to make sure that the way we implement BKT with pyAgrum corresponds to the values obtained by pyBKT. 

In [1]:
import sys
sys.path.append("/Users/olivier/PycharmProjects/bayesian-kst/")  # for mac

import pyAgrum as gum
from kgraph.expert_layer.domain import Domain
from kgraph.expert_layer.knowledge_components import KnowledgeComponent
from kgraph.expert_layer.link import Link
from kgraph.resources_layer.exercise import Exercise
from kgraph.learner_layer.answer import LearnerAnswer
from kgraph.learner_layer.learner import Learner
from kgraph.learner_layer.learner_pool import LearnerPool
from kgraph.helpers.truthtable import truthtable
from math import floor
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.dynamicBN as gdyn
import random
import itertools
import numpy as np
import tqdm

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold

def get_strongest_folds(full, axis="user_id", nb_folds=5):
    all_elements = full[axis].unique()

    kfold = KFold(nb_folds, shuffle=True)
    folds = []
    for i, (train, test) in enumerate(kfold.split(all_elements)):
        list_of_test_ids = []
        for element_id in test:
            list_of_test_ids += list(full.query(f'{axis} == {all_elements[element_id]}').index)
        folds.append(np.array(list_of_test_ids))
    
    return folds

def mae(true_vals, pred_vals):
    """ Calculates the mean absolute error. """
    return np.mean(np.abs(true_vals - pred_vals))


## Define the domain knowledge model

In [2]:

# we define the KCs

KC_A = KnowledgeComponent(55365, "Déterminer l'appartenance d'un nombre réel à un intervalle fini")
KC_B = KnowledgeComponent(55363, "Déterminer l'appartenance d'un nombre réel à un intervalle infini")
KC_C = KnowledgeComponent(55364, "Déterminer l'appartenance d'un nombre réel à un intervalle simple")
KC_D = KnowledgeComponent(50988, "Déterminer l'appartenance d'un nombre réel à une intersection d'intervalles de R")
KC_E = KnowledgeComponent(50989, "Déterminer l'appartenance d'un nombre réel à une réunion d'intervalles de R")

A_2_C = Link(source=KC_A, target=KC_C)
B_2_C = Link(source=KC_B, target=KC_C)
C_2_D = Link(source=KC_C, target=KC_D)
C_2_E = Link(source=KC_C, target=KC_E)
domain = Domain([KC_A, KC_B, KC_C, KC_D, KC_E], [A_2_C, B_2_C, C_2_D, C_2_E])

params = {"slip": .01, "guess":.01}

# there are 5 exercises corresponding to KC A
ex_A_1 = Exercise(237957, KC_A, "qcm", ex_content="", params=params)
ex_A_2 = Exercise(237958, KC_A, "qcm", ex_content="", params=params)
ex_A_3 = Exercise(237959, KC_A, "qcm", ex_content="", params=params)
ex_A_4 = Exercise(237960, KC_A, "qcm", ex_content="", params=params)
ex_A_5 = Exercise(237961, KC_A, "qcm", ex_content="", params=params)

# there are also 5 exercises corresponding to KC B
ex_B_1 = Exercise(237947, KC_B, "qcm", ex_content="", params=params)
ex_B_2 = Exercise(237948, KC_B, "qcm", ex_content="", params=params)
ex_B_3 = Exercise(237949, KC_B, "qcm", ex_content="", params=params)
ex_B_4 = Exercise(237950, KC_B, "qcm", ex_content="", params=params)
ex_B_5 = Exercise(237951, KC_B, "qcm", ex_content="", params=params)

ex_C_1 = Exercise(237952, KC_C, "qcm", ex_content="", params=params)
ex_C_2 = Exercise(237953, KC_C, "qcm", ex_content="", params=params)
ex_C_3 = Exercise(237954, KC_C, "qcm", ex_content="", params=params)
ex_C_4 = Exercise(237955, KC_C, "qcm", ex_content="", params=params)
ex_C_5 = Exercise(237956, KC_C, "qcm", ex_content="", params=params)


ex_D_1 = Exercise(225183, KC_D, "qcm", ex_content="", params=params)
ex_D_2 = Exercise(225184, KC_D, "qcm", ex_content="", params=params)
ex_D_3 = Exercise(225185, KC_D, "qcm", ex_content="", params=params)
ex_D_4 = Exercise(225186, KC_D, "qcm", ex_content="", params=params)
ex_D_5 = Exercise(225187, KC_D, "qcm", ex_content="", params=params)

ex_E_1 = Exercise(225165, KC_E, "qcm", ex_content="", params=params)
ex_E_2 = Exercise(225166, KC_E, "qcm", ex_content="", params=params)
ex_E_3 = Exercise(225167, KC_E, "qcm", ex_content="", params=params)
ex_E_4 = Exercise(225168, KC_E, "qcm", ex_content="", params=params)
ex_E_5 = Exercise(225169, KC_E, "qcm", ex_content="", params=params)

def get_KC_from_exercise_id(exercise_id):
    if exercise_id in range(237957, 237962):
        return KC_A
    elif exercise_id in range(237947, 237952):
        return KC_B
    elif exercise_id in range(237952, 237957):
        return KC_C
    elif exercise_id in range(225183, 225188):
        return KC_D
    else:
        return KC_E
    
learner_pool = LearnerPool(domain, {})

## Import the data

In [3]:
import pandas as pd

df = pd.read_csv("5_KCs_example_data.csv")

print(df)

folds = get_strongest_folds(df, "user_id", 2)
test_ids = folds[0]

train_ids = list(set(list(df.index.values)) - set(test_ids))

df_train = df[df.index.isin(train_ids)]
df_test = df[df.index.isin(test_ids)]

         idx  doc_id  exercise_id  evaluation_id  success  user_id  \
0          1   50988       225183      109276367        0   757204   
1          2   50988       225183      109293461        1  2052585   
2          3   50988       225183      109293517        1  2052585   
3          4   50988       225183      109293574        1  2052585   
4          5   50988       225183      109307385        1  1896564   
...      ...     ...          ...            ...      ...      ...   
42478  42479   55365       237961      151659716        1  1278392   
42479  42480   55365       237961      151664581        1  3926123   
42480  42481   55365       237961      151667070        1  3275043   
42481  42482   55365       237961      151667191        1  3275043   
42482  42483   55365       237961      151669835        1  1699544   

                 createdAt  
0      2019-08-12 12:57:49  
1      2019-08-12 19:18:03  
2      2019-08-12 19:20:45  
3      2019-08-12 19:23:40  
4      2019-08

## Learn parameters with pyBKT
We learn the parameters of the bayesian network with pyBKT (EM algorithm) on the train dataset. 

In [4]:
from pyBKT.models import Model

# Initialize the model with an optional seed
model = Model(seed = 42, num_fits = 1)
defaults = {'order_id': 'idx', 'skill_name': 'doc_id', 'correct': 'success'}

model.fit(data = df_train, defaults = defaults)

for kc in learner_pool.get_knowledge_components():
    learner_pool.set_learn(kc, model.params().loc[f'{kc.id}', 'learns', 'default'].value)
    learner_pool.set_prior(kc, model.params().loc[f'{kc.id}', 'prior', 'default'].value)
    learner_pool.set_slip(kc, model.params().loc[f'{kc.id}', 'slips', 'default'].value)
    learner_pool.set_guess(kc, model.params().loc[f'{kc.id}', 'guesses', 'default'].value)
    learner_pool.set_forget(kc, model.params().loc[f'{kc.id}', 'forgets', 'default'].value)


## Check prediction performance of pyBKT model
We then check how the prediction is performed with pyBKT and the learned parameters.

In [5]:
print('ACC', model.evaluate(data = df_test, metric = 'accuracy'))
print('AUC', model.evaluate(data = df_test, metric = 'auc'))
print('MAE', model.evaluate(data = df_test, metric = mae))

ACC 0.7375482534601262
AUC 0.7018746632618859
MAE 0.3626989069046264


## Check prediction performance of BKT with pyAgrum library
We setup the same process for BKT with pyAgrum, with parameters computed thanks to pyBKT. 

In [6]:
def get_bkt_net(learner, kc, n_eval):
    bkt_net = gum.BayesNet('BKT')
    learn = learner.learner_pool.get_learn(kc)
    prior = learner.learner_pool.get_prior(kc)
    guess = learner.learner_pool.get_guess(kc)
    slip = learner.learner_pool.get_slip(kc)
    forget = learner.learner_pool.get_forget(kc)
    bkt_net.add(gum.LabelizedVariable(f"({kc.name})0", '', 2))
    bkt_net.cpt(f"({kc.name})0").fillWith([1-prior, prior])
    for i in range(1, n_eval+1):
        bkt_net.add(gum.LabelizedVariable(f"({kc.name}){i}", '', 2))
        bkt_net.add(gum.LabelizedVariable(f"eval({kc.name}){i}", '', 2))
        bkt_net.addArc(*(f"({kc.name}){i}", f"eval({kc.name}){i}"))
        bkt_net.cpt(f"eval({kc.name}){i}")[{f"({kc.name}){i}": False}] = [1-guess, guess]
        bkt_net.cpt(f"eval({kc.name}){i}")[{f"({kc.name}){i}": True}] = [slip, 1-slip]
        bkt_net.addArc(*(f"({kc.name}){i-1}", f"({kc.name}){i}"))
        bkt_net.cpt(f"({kc.name}){i}")[{f"({kc.name}){i-1}": False}] = [1-learn, learn]
        bkt_net.cpt(f"({kc.name}){i}")[{f"({kc.name}){i-1}": True}] = [forget, 1-forget]

    return bkt_net

def evaluate_learner_bkt(learner, evaluations):
    n_eval = len(evaluations)
    evaluated_kc = evaluations[0][0]  # we suppose that evaluated_kc is the same for all evaluations
    floor_idx = 0
    expected_values = []
    predicted_values = []
    for i in range(floor_idx, n_eval):
        bn = get_bkt_net(learner, evaluated_kc, i+1)
        ie=gum.LazyPropagation(bn)
        ie.setEvidence({f"eval({evaluations[j][0].name}){j+1}": evaluations[j][1] for j in range(i)})
        ie.makeInference()
        predicted_values.append(ie.posterior(f"eval({evaluations[i][0].name}){i+1}")[:][1])
        expected_values.append(evaluations[i][1])

    return (expected_values, predicted_values)

In [7]:
### Score with pyAgrum

expected_values = []
predicted_values = []

for kc_id in tqdm.tqdm(df_test["doc_id"].unique()):
    kc_evals = df_test[df_test["doc_id"] == kc_id]
    for learner_id in kc_evals["user_id"].unique():
        learner = Learner(learner_id, learner_pool)
        learner_evals = kc_evals[kc_evals["user_id"] == learner_id]
        answers = [[get_KC_from_exercise_id(row["exercise_id"]), row["success"]] 
                   for i, row in learner_evals.iterrows()]
        n_eval = len(answers)
        floor_idx = 0
        exp_vals, pred_vals = evaluate_learner_bkt(learner, answers)
        predicted_values = np.concatenate((predicted_values, pred_vals))
        expected_values = np.concatenate((expected_values, exp_vals))

print('ACC', accuracy_score(expected_values, [1 if x>.5 else 0 for x in predicted_values]))
print('AUC', roc_auc_score(expected_values, predicted_values))
print('MAE', mae(expected_values, predicted_values))


100%|██████████| 5/5 [00:33<00:00,  6.79s/it]

ACC 0.7375482534601262
AUC 0.702061189270409
MAE 0.3638476046491296


Conclusion: same results but a way longer compute time